<a href="https://colab.research.google.com/github/drawoliver/FitDSFCurves/blob/main/FitDSFCurves.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FitDSFCurves**







**A python jiffy to fit DSF data with a modified Boltzmann equation to determine a value for Tm and slope.**

For 'simple' use, please choose 'Run all' option, from the Runtime drop down menu

— — — — — — — — — —

Run the first cell to install lmfit and seaborn packages.

Note: you only need to do this once to 'intialise' the program.

In [ ]:
#@title Initialisation

# Run once initialisation (lmfit, seaborn)
%pip install lmfit
%pip install seaborn

The next cell imports various packages needed to run the program, loads dependences and then asks the user to upload a .csv file.

NOTE: the first column should contain the temperature data. The remaining columns should contain the melting data [one column per individual experimental repeat].

In [2]:
#@title Dependencies, Upload .csv and Drop Headers

# Setup requirements
import io
import math
import pandas as pd
import numpy as np
import sympy
import matplotlib.pyplot as plt
import seaborn as sns
import lmfit
from scipy.optimize import curve_fit
from google.colab import files
from google.colab import output

# Upload .csv file and eliminate NaN and column headers
uploaded = files.upload()

# Upload .csv file, drop headers and remove NaN values
filename = next(iter(uploaded))
raw_data = pd.read_csv(filename, header=None, skiprows=1)
raw_data = rawdata.dropna(axis='rows')


The next cell does the curve fitting for each individual dataset and plots a graph.


In [ ]:
#@title Fit and Plot Data

def spacer ():
  print ("")
  print ("— — — — —")
  print ("")
  return

spacer()

# choose lower and upper ranges for fit
lower_range = float(input ("Please enter lowest temperature for fitting, enter 0 to use full input range "))
upper_range = float(input ("Please enter highest temperature for fitting, enter 0 to use full input range: "))

if lower_range == 0:
  data_lower = raw_data
else:
  data_lower = raw_data[rawdata[0] >= lower_range]

if upper_range == 0:
  data_upper = data_lower
else:
  data_upper = data_lower[data_lower[0] <= upper_range]

data = data_upper

spacer()

# Create x and y-data variables from uploaded .csv file (y_data1 for column 1, y_data2 for column2 etc)
num_columns = len(data.columns)
x_data = data.iloc[:, 0]

for i in range(1, num_columns):
    globals()[f"y_data{i}"] = data.iloc[:, i]


# Equation for fitting data
def user_defined_equation_1(x, slope_native, intercept_native, slope_denatured, intercept_denatured, melting_temp, slope_midpoint):
  return ((slope_native * x + intercept_native) + (((slope_denatured * x + intercept_denatured)-(slope_native * x + intercept_native)/(1 + (np.exp((melting_temp - x) / slope_midpoint))))))

# Number of iterations
max_iterations = 1000

# Set initial guess
initial_guess = [0,0,0,1,35,2]

# Perform iterative fitting with Levenberg-Marquardt

for i in range (1, num_columns):

  input_y_data = globals()[f"y_data{i}"]

  for iteration in range(max_iterations):

    # Use curve_fit with the current initial guess and 'lm' method
    fit_params, covariance = curve_fit(user_defined_equation_1, x_data, input_y_data, p0=initial_guess, method='trf', bounds=((-np.inf,-np.inf,-np.inf,0.99,20,-np.inf), (np.inf,np.inf,np.inf,1.0,120,np.inf)) )
    # Generate fitted y values using the fitted parameters
    y_fit = user_defined_equation_1(x_data, *fit_params)

    # Check for convergence (you can customize this condition based on your needs)
    if np.all(np.abs(np.array(initial_guess) - np.array(fit_params)) < 1e-6):
        print(f'Data in column {i} converged after {iteration + 1} iterations.')
        break

    # Update initial guess for the next iteration
    initial_guess = fit_params

  globals()[f"Tm_column{i}"] = fit_params[4]
  globals()[f"Slope_column{i}"] = fit_params[5]

  # Initialize plot
  sns.set_theme()
  fig, ax = plt.subplots()
  # Plot data
  ax.scatter(x_data, input_y_data, label='Data', color='black')
  plt.xlabel('Temperature ˚C')
  plt.ylabel('Fraction unfolded')
  ax.plot(x_data, y_fit, color='red')
  plt.show()

  # Print the final values for each parameter
  print("Fitted Values:")
  print("Tm:",fit_params[4])
  print("Slope:",fit_params[5])
  print ("")
  #print("Native slope:",fit_params[0])
  #print("Native intercept:",fit_params[1])
  #print("Denatured slope:",fit_params[2])
  #print("Denatured intercept", fit_params[3])




The next cell creates a short summary for the fitted data.  It also reports average values (but this is only valid if the input data represent true experimental replicates).

In [ ]:
#@title Create Summary

# Create Summary
sum_tm = 0

for i in range (1, num_columns):
  result_float = round(float(globals()[f"Tm_column{i}"]),2)
  print("Tm for column",i,":", result_float)
  sum_tm = sum_tm + result_float

average_tm = sum_tm / (num_columns-1)
result_average_tm = round(average_tm, 2)
print("Average Tm:", result_average_tm)

print("")

sum_slope = 0

for i in range (1, num_columns):
  result_float = round(float(globals()[f"Slope_column{i}"]),2)
  print("Slope for column",i,":", result_float)
  sum_slope = sum_slope + result_float

average_slope = sum_slope / (num_columns-1)
result_average_slope = round(average_slope,2)
print ("Average Slope:", result_average_slope)
